In [51]:
2+2

4

### Importing Libreries

In [118]:
import pandas as pd
import os
from tqdm import tqdm
from dotenv import load_dotenv
load_dotenv()

True

### Load the dataset

In [109]:
df = pd.read_csv("E:/data science/FreeLance_proj/assgin/documents/Assignment.csv", encoding='latin-1')
df.head(5)

,Article
0,"Retailers, the makers of foods marketed for we..."
1,"Move over, Ozempic  theres a new drug in tow..."
2,Sept 14 (Reuters) - Bristol Myers Squibb (BMY....
3,Austin Wolcott was 18 years old and pretty sur...
4,"Cancer, often referred to as the emperor of a..."


### Implementing Groq API for Response Generation with LLAMA3

In [119]:
from groq import Groq

client = Groq( api_key = os.environ["GROQ_API_KEY"])

## Functions for doing the operations

In [111]:
class article_inshight:
    ## Function for cleaning article
    def clean_article(prompt):
        res = client.chat.completions.create(
            messages= [
                    {"role": "system", "content": """
                        You are an advace AI article cleaner, your role is to,
                        Remove unnecessary words and phrases:  Focus on the essential information. 
                        Shorten sentences: Aim for a more direct and impactful style. Length of the res should be within 700 words.
                        Replace complex words with simpler alternatives: Keep it accessible to a wider audience.
                        Use strong verbs and active voice: Make the writing more engaging and direct.
                        Preserve the key message: The cleaned article should convey the same main points as the original.
                    """}, 
                    {"role": "user", "content": prompt}
                ], 
                model="llama3-8b-8192"
            )
        return res.choices[0].message.content 

    # Function for detecting sentiment
    def mood_detect(prompt):
        res = client.chat.completions.create(
            messages= [
                    {"role": "system", "content": """
                        You are an advace AI emotion detector, your role is to,
                        check the given article summary and tell the mood of that article. 
                        Your mood checker should be either **Neutral** or **Negetive** or **Positive**.
                        Your response should be within 1 word, which the Mood of the article (Neutral / Negetive / Positive)
                    """}, 
                    {"role": "user", "content": prompt}
                ], 
                model="llama3-8b-8192"
            )
        return res.choices[0].message.content 

    # Function for finding connection themes
    def connection_detect(prompt):
        res = client.chat.completions.create(
            messages= [
                    {"role": "system", "content": """
                        You are an advace AI investigetor, your role is to,
                        Provide a concise list of the identified common themes, using clear and descriptive language.
                        Briefly explain how each theme is supported by evidence from the articles. 
                        Feel free to provide any additional insights or observations you have regarding the articles and their common themes. 
                        Your each for every articles should be within 40 words
                    """}, 
                    {"role": "user", "content": f"You will be given all the articles one by one: \n {prompt}"}
                ], 
                model="mixtral-8x7b-32768"
            )
        return res.choices[0].message.content 

    # Function for detecting the aspect
    def aspect_detect(prompt):
        res = client.chat.completions.create(
            messages= [
                    {"role": "system", "content": """
                        You are an advace AI sentiment explorer, your role is to,
                        check the given article and explores sentiment towards different aspects ofa news story. 
                        some examples are:
                            article - "New solar panel developed, but critics say it's too expensive.", aspect - "Innovation Cost"
                            article - "City council approves new park plan, but some residents are concerned about traffic.", aspect - Overall Plan Traffic Impact
                        Your task is to give only the aspect nothing else within 20 words.
                    """}, 
                    {"role": "user", "content": prompt}
                ], 
                model="llama3-8b-8192"
            )
        return res.choices[0].message.content

### Getting the response generation

In [125]:
cleaned_art = []
mood_art = []
aspect = []

In [126]:
## Cleaing the article
for i in tqdm(df.values, desc="Cleaning Articles"):
    cleaned_art.append(article_inshight.clean_article(str(i)))

print(len(cleaned_art))

## Detecting the Mood
for cl in tqdm(cleaned_art[::], desc="Detecting Mood"):
    mood_art.append(article_inshight.mood_detect(cl))

mood_arti = [mood.replace('**', '') for mood in mood_art]

## Detecing the aspect
for ia in tqdm(cleaned_art, desc="Detecting Aspect"):
    aspect.append(article_inshight.aspect_detect(ia))

## Finding the Connection
res = article_inshight.connection_detect(cleaned_art)



Cleaning Articles: 100%|██████████| 25/25 [00:28<00:00,  1.14s/it]


25


Detecting Aspect: 100%|██████████| 25/25 [01:14<00:00,  2.97s/it]


### Final Output

In [127]:
data = {
    "Article (Cleaned)": cleaned_art,
    "Aspect": aspect,
    "Mood Rating": mood_arti
}

df_new = pd.DataFrame(data)
df_new.head()


,Article (Cleaned),Aspect,Mood Rating
0,"Here is the rewritten article, incorporating t...","""Industry Move""",Neutral
1,Here's the rewritten article:\n\nEli Lilly's n...,Drugs,Neutral
2,Bristol Myers Squibb Plans to Double Clinical ...,Innovation Strategy,Positive
3,Here is the rewritten article:\n\nAustin Wolco...,Aspect: Innovation Cost,Positive
4,Here is the cleaned article:\n\nCancer: The Fo...,Innovation,Positive


In [131]:
df_new.to_csv('E:/data science/FreeLance_proj/assgin/documents/output.csv', index=False)

In [129]:
print(res)

1. Ozempic and the weight loss drug industry: Articles discuss the rise of diabetes and weight loss drugs like Ozempic, a GLP-1 drug that mimics a glucagon-like peptide regulating appetite in the brain, leaving users feeling full for hours. Companies are looking for ways to capitalize on this trend, as Ozempic has led to a decline in earnings for some diet product companies.

2. Medicare price negotiations and drug costs: Articles mention the first list of Medicare price negotiations, which includes 10 medicines subject to negotiations to reduce costs. Drugmakers have pushed back against the plan, and some Republicans have criticized it as government overreach.

3. #Girlboss movement and women in leadership: Articles discuss the #Girlboss movement and its impact on companies with women in leadership roles. Some argue the term is offensive and diminutive, while others highlight the struggles of companies like Outdoor Voices and Man Repeller, which faced scandals and lawsuits with men at

***